In [3]:
### Check the correct virtual enviornment

import sys
sys.prefix

'c:\\Users\\Admin\\Documents\\GEN_AI\\Langchain\\3-Conv_History\\.venv'

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key

'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'

In [5]:
from langchain_groq import ChatGroq
model = ChatGroq(model='gemma2-9b-it', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B82FCBBD50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B82EAFD350>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Documentation on Human , System Message

https://python.langchain.com/api_reference/core/messages.html

https://python.langchain.com/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html

https://python.langchain.com/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html

https://python.langchain.com/api_reference/core/messages/langchain_core.messages.ai.AIMessage.html

In [7]:

from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content='Hi my name is Ankur and I am the chief AI engineer.')])

AIMessage(content="Hello Ankur, it's nice to meet you! As a large language model, I'm always eager to learn from experienced AI engineers like yourself. \n\nWhat are you working on these days?  Is there anything I can help you with?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 23, 'total_tokens': 78, 'completion_time': 0.1, 'prompt_time': 0.000153779, 'queue_time': 0.01408276, 'total_time': 0.100153779}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3a939c9c-981f-4ab4-bfda-f35969c3fcd7-0', usage_metadata={'input_tokens': 23, 'output_tokens': 55, 'total_tokens': 78})

### Scenario 1 

Creating a simple chatbot where messages are passed as sequence and the chat model is able to remember the information provided

In [8]:
from langchain_core.messages import AIMessage

In [9]:
messages = [HumanMessage(content='Hi my name is Ankur and I am the chief AI engineer.'),
            AIMessage(content="Hello Ankur, it's nice to meet you! As a large language model, I'm always eager to learn from experienced AI engineers like yourself. \n\nWhat are you working on these days?  Is there anything I can help you with?\n"),
            HumanMessage("What is my name and what I do?")]

In [10]:
model.invoke(messages)

AIMessage(content="You told me your name is Ankur and that you are the chief AI engineer. 😊 \n\nIs there anything else you'd like to know or discuss?  Perhaps you'd like to explore some AI concepts, brainstorm ideas for a project, or just chat about the field of AI?  I'm here to help!  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 95, 'total_tokens': 167, 'completion_time': 0.130909091, 'prompt_time': 0.002913471, 'queue_time': 0.012643946999999999, 'total_time': 0.133822562}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-15ea4e67-d440-4557-ab55-ca0cf36fb887-0', usage_metadata={'input_tokens': 95, 'output_tokens': 72, 'total_tokens': 167})

### Scenario 2

- Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

Documentation for 


- ChatMessageHistory https://python.langchain.com/api_reference/community/chat_message_histories.html


- BaseChatMessageHistory   https://python.langchain.com/api_reference/core/chat_history/langchain_core.chat_history.BaseChatMessageHistory.html


- RunnableWithMessageHistory   https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory ### Chat message history stores a history of the message interactions in a chat.


from langchain_core.chat_history import BaseChatMessageHistory   ###  Abstract base class for storing chat message history.


from langchain_core.runnables.history import RunnableWithMessageHistory  ### RunnableWithMessageHistory wraps another Runnable and manages the chat message history for it; it is responsible for reading and updating the chat message history

When multiple users interact with the chat model how we will make sure that one session is completly different from another sesssion ?

We will create a function as below -

1) It accepts session ID as input. It will be used to distinguish one session from another
2) It returns the object of type BaseChatMessageHistory

In [15]:
store = {}

### function that retrives chat history for a specific session id

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [16]:
### Interact with LLM model based on chat history

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [17]:
### Define a config parameter that defines the session id for a perticular session

### Here we have hardcoded our session id as 'chat1'

config = {'configurable' : {'session_id' : 'chat1'}}

In [18]:
messages = [HumanMessage(content = "My name is Ashish and I am an DevOps engineer")]
with_message_history.invoke(messages, config=config)

AIMessage(content="Hello Ashish, it's nice to meet you!\n\nAs a DevOps engineer, I imagine you have a lot of interesting projects and challenges you work on. What are some of the things you enjoy most about your work?  \n\nI'm always eager to learn more about different roles in the tech industry. Perhaps you could tell me a bit about your typical day or some of the tools and technologies you use?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 20, 'total_tokens': 108, 'completion_time': 0.16, 'prompt_time': 8.1039e-05, 'queue_time': 0.022787434999999998, 'total_time': 0.160081039}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b0d2d53c-1937-4c0b-9883-ebff1de5f0c5-0', usage_metadata={'input_tokens': 20, 'output_tokens': 88, 'total_tokens': 108})

In [19]:
### Lets ask the chat model another question with respect to same session id

with_message_history.invoke([HumanMessage(content='What is my name and what I do?')], config=config)

AIMessage(content="You said your name is Ashish and that you are a DevOps engineer. \n\nIs there anything else you'd like to tell me about yourself or ask me? I'm ready for your next question! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 125, 'total_tokens': 173, 'completion_time': 0.087272727, 'prompt_time': 0.004427463, 'queue_time': 0.017586714, 'total_time': 0.09170019}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ca963389-91b2-46df-b7bf-685a924ac73c-0', usage_metadata={'input_tokens': 125, 'output_tokens': 48, 'total_tokens': 173})

In [20]:
### Lets create a new sesson id

config_2 = {'configurable' : {'session_id' : 'chat2'}}

In [21]:
messages = [HumanMessage(content='My name is Suresh and I am a CA')]
with_message_history.invoke(messages, config=config_2)

AIMessage(content="It's nice to meet you, Suresh!\n\nThat's great that you're a CA (Chartered Accountant). It's a demanding but rewarding profession. \n\nIs there anything I can help you with today? Perhaps you have a quick accounting question, need to brainstorm ideas for a project, or just want to chat about your work?  \n\nI'm here to assist in any way I can.  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 18, 'total_tokens': 109, 'completion_time': 0.165454545, 'prompt_time': 9.3919e-05, 'queue_time': 0.021384046, 'total_time': 0.165548464}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-47c696fc-bd68-4b97-9be4-2cdab4bc5c6b-0', usage_metadata={'input_tokens': 18, 'output_tokens': 91, 'total_tokens': 109})

In [22]:
### Lets ask the question to the chat model with respect to 'chat2' session id

with_message_history.invoke([HumanMessage(content='What is my name and what I do?')], config=config_2)

AIMessage(content="You are Suresh, and you are a CA (Chartered Accountant). 😊  \n\nIs there anything else you'd like to tell me about yourself, or would you like to discuss something related to your work as a CA?  I'm here to listen and help!  \n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 125, 'total_tokens': 186, 'completion_time': 0.110909091, 'prompt_time': 0.003907445, 'queue_time': 0.014067321, 'total_time': 0.114816536}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5649e8d1-2bad-4ec7-97c7-a08d00af823c-0', usage_metadata={'input_tokens': 125, 'output_tokens': 61, 'total_tokens': 186})

In [23]:
### Lets create a new session id and ask the same question as previously asked

config_3 = {'configurable' : {'session_id' : 'chat3'}}

In [24]:
with_message_history.invoke([HumanMessage(content='What is my name and what I do?')], config=config_3)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name or what you do.\n\nIf you'd like to tell me, I'm happy to learn! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 18, 'total_tokens': 61, 'completion_time': 0.078181818, 'prompt_time': 0.00010439, 'queue_time': 0.016184319, 'total_time': 0.078286208}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-94695679-7904-4a9c-b042-b91f58c3fa1e-0', usage_metadata={'input_tokens': 18, 'output_tokens': 43, 'total_tokens': 61})